## Scrapping bantennews.co.id

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
jumlah_index = 27
threads_link = []
links = []
results = []
threads = []
keywords='ganjar'

In [3]:
def scrape_links(page_number,keywords):
    url=f"https://www.bantennews.co.id/page/{page_number}/?s={keywords}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3',{"class": "entry-title td-module-title"})
    
    page_links = []
    for article in articles:
        link_element = article.find('a')
        if link_element is not None:
            link = link_element.get('href')
            if link is not None:
                page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 11
Scraped 10 links from page 3
Scraped 10 links from page 14
Scraped 10 links from page 15
Scraped 10 links from page 20
Scraped 10 links from page 21
Scraped 10 links from page 18
Scraped 10 links from page 4
Scraped 10 links from page 17
Scraped 10 links from page 16
Scraped 10 links from page 9
Scraped 10 links from page 1
Scraped 10 links from page 8
Scraped 10 links from page 10
Scraped 10 links from page 24
Scraped 10 links from page 19
Scraped 10 links from page 22
Scraped 10 links from page 7
Scraped 10 links from page 13
Scraped 10 links from page 2
Scraped 10 links from page 25
Scraped 10 links from page 12
Scraped 10 links from page 6
Scraped 4 links from page 27
Scraped 10 links from page 5
Scraped 10 links from page 26
Scraped 10 links from page 23
Total Links: 264


In [5]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "entry-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            # author_elem = soup.find('div', {"class": "tags-wrapper"})
            # if author_elem:
            #     author_text = author_elem.find('span')
            #     author_text = author_text.text.split(':')[1]
            # else:
            author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('time', {"class": "entry-date updated td-module-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('li', {"class": "entry-category"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "td-post-content td-pb-padding-side"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.co\.id/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  264


,title,keywords,author,category,date,content,nama_berita,region,link
0,Projo Condong Dukung Gibran Jadi Cawapres Prabowo,ganjar,Author not found,Pemilu 2024,"Minggu 1 Okt 2023, 06:13 WIB","SERANG – Ketua Bapilpres Relawan Projo, Panel...",bantennews,banten,https://www.bantennews.co.id/projo-condong-duk...
1,Ini Deretan Partai yang Dukung Prabowo Subiant...,ganjar,Author not found,Pemilu 2024,"Senin 31 Jul 2023, 06:10 WIB",SERANG – Partai Bulan Bintang (PBB) mendeklara...,bantennews,banten,https://www.bantennews.co.id/ini-deretan-parta...
2,Ini Hasil Survei Terbaru Elektabilitas Capres ...,ganjar,Author not found,Pemilu 2024,"Minggu 30 Jul 2023, 12:14 WIB",SERANG – Ganjar Pranowo mengungguli elektabili...,bantennews,banten,https://www.bantennews.co.id/ini-hasil-survei-...
3,Ada Efek Jokowi di Balik ‘Musuh-Musuh’ yang Ki...,ganjar,Author not found,Pemilu 2024,"Sabtu 22 Jul 2023, 06:07 WIB",SERANG – Pengamat politik Ujang Komaruddin mel...,bantennews,banten,https://www.bantennews.co.id/ada-efek-jokowi-d...
4,Apa Untungnya Jika Prabowo Pinang Gibran Jadi ...,ganjar,Author not found,Pemilu 2024,"Rabu 11 Okt 2023, 07:10 WIB",SERANG – Bakal Calon Presiden (Bacapres) Prabo...,bantennews,banten,https://www.bantennews.co.id/apa-untungnya-jik...
5,Sandiaga Uno Ikhlas Jika Gagal Jadi Cawapres G...,ganjar,Author not found,Pemilu 2024,"Senin 7 Agu 2023, 12:09 WIB",SERANG – Ketua Badan Pemenangan Pemilu (Bappil...,bantennews,banten,https://www.bantennews.co.id/sandiaga-uno-ikhl...
6,Demokrat Dorong Anies Baswedan Umumkan Bakal C...,ganjar,Author not found,Pemilu 2024,"Selasa 6 Jun 2023, 10:14 WIB",JAKARTA – DPP Partai Demokrat memberikan warni...,bantennews,banten,https://www.bantennews.co.id/demokrat-dorong-a...
7,Jusuf Kalla Tak Akan Gabung Timses Capres-Cawa...,ganjar,Author not found,Pemilu 2024,"Rabu 11 Okt 2023, 03:19 WIB","SERANG – Presiden ke-10 dan ke-12 RI, Jusuf Ka...",bantennews,banten,https://www.bantennews.co.id/jusuf-kalla-tak-a...
8,Polres Serang Bongkar Home Industry Pembuatan ...,ganjar,Author not found,Hukum,"Senin 9 Okt 2023, 14:09 WIB",SERANG – Berhasil membongkar “home Industry” t...,bantennews,banten,https://www.bantennews.co.id/polres-serang-bon...
9,Isu Gibran Jadi Cawapres Prabowo Disebut Dimun...,ganjar,Author not found,Pemilu 2024,"Jumat 29 Sep 2023, 05:13 WIB",SERANG – Sekretaris Jenderal DPP PDI Perjuanga...,bantennews,banten,https://www.bantennews.co.id/isu-gibran-jadi-c...


In [8]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 263


,title,keywords,author,category,date,content,nama_berita,region,link
0,Projo Condong Dukung Gibran Jadi Cawapres Prabowo,ganjar,Author not found,Pemilu 2024,"Minggu 1 Okt 2023, 06:13 WIB","SERANG – Ketua Bapilpres Relawan Projo, Panel...",bantennews,banten,https://www.bantennews.co.id/projo-condong-duk...
1,Ini Deretan Partai yang Dukung Prabowo Subiant...,ganjar,Author not found,Pemilu 2024,"Senin 31 Jul 2023, 06:10 WIB",SERANG – Partai Bulan Bintang (PBB) mendeklara...,bantennews,banten,https://www.bantennews.co.id/ini-deretan-parta...
2,Ini Hasil Survei Terbaru Elektabilitas Capres ...,ganjar,Author not found,Pemilu 2024,"Minggu 30 Jul 2023, 12:14 WIB",SERANG – Ganjar Pranowo mengungguli elektabili...,bantennews,banten,https://www.bantennews.co.id/ini-hasil-survei-...
3,Ada Efek Jokowi di Balik ‘Musuh-Musuh’ yang Ki...,ganjar,Author not found,Pemilu 2024,"Sabtu 22 Jul 2023, 06:07 WIB",SERANG – Pengamat politik Ujang Komaruddin mel...,bantennews,banten,https://www.bantennews.co.id/ada-efek-jokowi-d...
4,Apa Untungnya Jika Prabowo Pinang Gibran Jadi ...,ganjar,Author not found,Pemilu 2024,"Rabu 11 Okt 2023, 07:10 WIB",SERANG – Bakal Calon Presiden (Bacapres) Prabo...,bantennews,banten,https://www.bantennews.co.id/apa-untungnya-jik...
5,Sandiaga Uno Ikhlas Jika Gagal Jadi Cawapres G...,ganjar,Author not found,Pemilu 2024,"Senin 7 Agu 2023, 12:09 WIB",SERANG – Ketua Badan Pemenangan Pemilu (Bappil...,bantennews,banten,https://www.bantennews.co.id/sandiaga-uno-ikhl...
6,Demokrat Dorong Anies Baswedan Umumkan Bakal C...,ganjar,Author not found,Pemilu 2024,"Selasa 6 Jun 2023, 10:14 WIB",JAKARTA – DPP Partai Demokrat memberikan warni...,bantennews,banten,https://www.bantennews.co.id/demokrat-dorong-a...
7,Jusuf Kalla Tak Akan Gabung Timses Capres-Cawa...,ganjar,Author not found,Pemilu 2024,"Rabu 11 Okt 2023, 03:19 WIB","SERANG – Presiden ke-10 dan ke-12 RI, Jusuf Ka...",bantennews,banten,https://www.bantennews.co.id/jusuf-kalla-tak-a...
8,Polres Serang Bongkar Home Industry Pembuatan ...,ganjar,Author not found,Hukum,"Senin 9 Okt 2023, 14:09 WIB",SERANG – Berhasil membongkar “home Industry” t...,bantennews,banten,https://www.bantennews.co.id/polres-serang-bon...
9,Isu Gibran Jadi Cawapres Prabowo Disebut Dimun...,ganjar,Author not found,Pemilu 2024,"Jumat 29 Sep 2023, 05:13 WIB",SERANG – Sekretaris Jenderal DPP PDI Perjuanga...,bantennews,banten,https://www.bantennews.co.id/isu-gibran-jadi-c...


In [9]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/bantennews_{keywords}_{current_datetime}.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../../tempat_hasil_daerah/bantennews_ganjar_2023-10-12_17-20-02.xlsx
